Selecting Traffic Congestion Data

In [ ]:
SELECT CURRENT_DATABASE(), CURRENT_SCHEMA();
SHOW DATABASES;

In [ ]:
USE DATABASE PROJECT_DB_FINAL;
USE SCHEMA PROCESSED_DATA;

In [ ]:
SELECT * FROM PROCESSED_DATA.TRAFFIC_CONGESTION_CAUSES LIMIT 5;


In [ ]:
CREATE OR REPLACE TABLE CLEANED_TRAFFIC AS
SELECT 
-- change all to uppercase
    UPPER(TRIM(COUNTY)) as COUNTY_NAME,
    UPPER(TRIM(STATE)) as STATE_ABBR,
    SUM("Incident (VHD)") as TOTAL_INCIDENT_CONGESTION,
    SUM("Weather (VHD)") as TOTAL_WEATHER_CONGESTION,
    SUM("Workzone (VHD)") as TOTAL_WORKZONE_CONGESTION,
    
    -- Create overall congestion score
    SUM("Incident (VHD)" + "Weather (VHD)" + "Workzone (VHD)" + "Recurrent (VHD)") as TOTAL_CONGESTION_SCORE
FROM TRAFFIC_CONGESTION_CAUSES
WHERE STATE IN ('AL','AR','FL','GA','LA','MS','NC','SC','TX','TN')
GROUP BY COUNTY_NAME, STATE_ABBR;


In [ ]:
--view new created table 
SELECT * FROM PROCESSED_DATA.CLEANED_TRAFFIC LIMIT 5

Selecting HPSA Data

In [ ]:
SELECT * FROM PROCESSED_DATA.HPSA_DATA_FIXED LIMIT 5

In [ ]:
CREATE OR REPLACE TABLE CLEANED_HPSA AS
SELECT 
    FIPS_CODE,
    HPSA_SCORE,
    DESIGNATION_TYPE,
    HPSA_DESIGNATION_POPULATION,
    PCT_POPULATION_BELOW_POVERTY,
    RURAL_STATUS,
    STATE_ABBR
FROM HPSA_DATA_FIXED
WHERE STATE_ABBR IN ('AL','AR','FL','GA','LA','MS','NC','SC','TX','TN')

In [ ]:
--view new table
SELECT * FROM PROCESSED_DATA.CLEANED_HPSA LIMIT 5

Selecting CDC SVI Data

In [ ]:
CREATE OR REPLACE TABLE CLEANED_CDC_SVI AS
SELECT 
     SUBSTRING(FIPS, 1, 5) as FIPS_CODE,
    -- Key SVI themes (for vulnerability)
    RPL_THEMES as OVERALL_SVI_SCORE,           -- Overall vulnerability
    RPL_THEME1 as SOCIOECONOMIC_VULNERABILITY, -- Poverty, unemployment, etc.
    RPL_THEME2 as HOUSEHOLD_VULNERABILITY,     -- Age 65+, disability, etc.
    RPL_THEME3 as MINORITY_STATUS,             -- Race, language barriers
    RPL_THEME4 as HOUSING_TRANSPORT,           -- Housingtype, vehicle access
    E_TOTPOP as TOTAL_POPULATION   
FROM CDC_SVI_SOUTH
WHERE ST_ABBR IN ('AL','AR','FL','GA','LA','MS','NC','SC','TX','TN')

In [ ]:
SELECT * FROM PROCESSED_DATA.CLEANED_CDC_SVI LIMIT 50

Some issues with CDC Places... plan to drop for  now

Selecting from NOAA Storms. The FIPS code is either state (2 numbers) or country (3 numbers) level which makes it difficult to keep consistent with the rest.

In [ ]:
CREATE OR REPLACE TABLE CLEANED_NOAA AS
SELECT 
    STATE,
    STATE_FIPS,
    CZ_FIPS,
    LPAD(STATE_FIPS::STRING, 2, '0') || LPAD(CZ_FIPS::STRING, 3, '0') AS         FIPS_CODE,
    CASE 
        WHEN EVENT_TYPE IN ('Hurricane','Tropical Storm','Flood','Tornado',
                           'Thunderstorm Wind','Hail','Lightning') THEN EVENT_TYPE
        ELSE 'Other Severe'
    END as SEVERE_EVENT_TYPE,
    COUNT(*) as EVENT_COUNT,
    SUM(
        CASE 
            -- Handle 'K' for thousands
            WHEN UPPER(DAMAGE_PROPERTY) LIKE '%K' THEN 
                TRY_CAST(REPLACE(UPPER(DAMAGE_PROPERTY), 'K', '') AS FLOAT) * 1000
            -- Handle 'M' for millions  
            WHEN UPPER(DAMAGE_PROPERTY) LIKE '%M' THEN
                TRY_CAST(REPLACE(UPPER(DAMAGE_PROPERTY), 'M', '') AS FLOAT) * 1000000
            -- Handle plain numbers
            ELSE TRY_CAST(DAMAGE_PROPERTY AS FLOAT)
        END
    ) as TOTAL_DAMAGE
FROM STORM_EVENTS_DETAILS
WHERE STATE IN ('ALABAMA','ARKANSAS','FLORIDA','GEORGIA',
               'LOUISIANA','MISSISSIPPI','NORTH CAROLINA','SOUTH CAROLINA',
               'TENNESSEE','TEXAS')
GROUP BY 
    STATE,
    STATE_FIPS,
    CZ_FIPS,
    SEVERE_EVENT_TYPE;

In [ ]:
SELECT * FROM PROCESSED_DATA.CLEANED_NOAA LIMIT 50

In [ ]:
CREATE OR REPLACE TABLE CLEANED_NOAA_AGGREGATED AS
SELECT 
    FIPS_CODE,
    STATE,
    COUNT(*) as TOTAL_EVENT_COUNT,
    SUM(TOTAL_DAMAGE) as TOTAL_DAMAGE_SUM,
    -- You might want to keep event type breakdowns in a separate column
    ARRAY_AGG(SEVERE_EVENT_TYPE) as EVENT_TYPES,
    ARRAY_AGG(EVENT_COUNT) as EVENT_COUNTS_BY_TYPE
FROM CLEANED_NOAA
GROUP BY FIPS_CODE, STATE;

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
df_svi = session.table('CLEANED_CDC_SVI')
df_svi = df_svi.to_pandas()
df_hpsa = session.table('CLEANED_HPSA')
df_hpsa = df_hpsa.to_pandas()
df_noaa = session.table('CLEANED_NOAA')
df_noaa = df_noaa.to_pandas()
df_traffic = session.table('CLEANED_TRAFFIC')
df_traffic = df_traffic.to_pandas()

In [ ]:
# See what columns are common between all
cols_svi = set(df_svi.columns)
cols_hpsa = set(df_hpsa.columns)
cols_noaa = set(df_noaa.columns)
cols_traffic = set(df_traffic.columns)

common_cols = cols_svi & cols_hpsa & cols_noaa
print(common_cols)


Join CDC, HPSA, and NOAA by FIPS 

In [ ]:
-- First see common FIPS code between all
WITH all_fips AS (
    SELECT 'HPSA' as source, FIPS_CODE FROM CLEANED_HPSA
    UNION ALL
    SELECT 'CDC_SVI' as source, FIPS_CODE FROM CLEANED_CDC_SVI
    UNION ALL
    SELECT 'NOAA' as source, FIPS_CODE FROM CLEANED_NOAA
),
counts AS (
    SELECT 
        COUNT(DISTINCT CASE WHEN source = 'HPSA' THEN FIPS_CODE END) as total_hpsa,
        COUNT(DISTINCT CASE WHEN source = 'CDC_SVI' THEN FIPS_CODE END) as total_cdc_svi,
        COUNT(DISTINCT CASE WHEN source = 'NOAA' THEN FIPS_CODE END) as total_noaa
    FROM all_fips
)
SELECT 
    total_hpsa,
    total_cdc_svi,
    total_noaa,
    (SELECT COUNT(*) FROM (
        SELECT FIPS_CODE FROM CLEANED_HPSA
        INTERSECT
        SELECT FIPS_CODE FROM CLEANED_CDC_SVI
    )) as hpsa_cdc_common,
    (SELECT COUNT(*) FROM (
        SELECT FIPS_CODE FROM CLEANED_HPSA
        INTERSECT
        SELECT FIPS_CODE FROM CLEANED_NOAA
    )) as hpsa_noaa_common,
    (SELECT COUNT(*) FROM (
        SELECT FIPS_CODE FROM CLEANED_CDC_SVI
        INTERSECT
        SELECT FIPS_CODE FROM CLEANED_NOAA
    )) as cdc_noaa_common,
    (SELECT COUNT(*) FROM (
        SELECT FIPS_CODE FROM CLEANED_HPSA
        INTERSECT
        SELECT FIPS_CODE FROM CLEANED_CDC_SVI
        INTERSECT
        SELECT FIPS_CODE FROM CLEANED_NOAA
    )) as all_three_common
FROM counts;

Should have 

In [ ]:
CREATE OR REPLACE TABLE JOINED_COUNTY_DATA AS
SELECT 
    h.FIPS_CODE,
    h.STATE_ABBR,
    
    -- HPSA Features (worst-case scenario)
    MAX(h.HPSA_SCORE) as MAX_HPSA_SCORE,
    AVG(h.HPSA_SCORE) as AVG_HPSA_SCORE,
    COUNT(DISTINCT h.DESIGNATION_TYPE) as NUM_DESIGNATION_TYPES,
    MAX(h.HPSA_DESIGNATION_POPULATION) as HPSA_POPULATION,
    MAX(h.PCT_POPULATION_BELOW_POVERTY) as POVERTY_RATE,
    MAX(h.RURAL_STATUS) as RURAL_STATUS,
    
    -- CDC SVI Features
    MAX(c.OVERALL_SVI_SCORE) as SVI_SCORE,
    MAX(c.SOCIOECONOMIC_VULNERABILITY) as SOCIOECONOMIC_SVI,
    MAX(c.HOUSEHOLD_VULNERABILITY) as HOUSEHOLD_SVI,
    MAX(c.MINORITY_STATUS) as MINORITY_SVI,
    MAX(c.HOUSING_TRANSPORT) as HOUSING_TRANSPORT_SVI,
    MAX(c.TOTAL_POPULATION) as TOTAL_POPULATION,
    
    -- NOAA Disaster Features
    SUM(n.EVENT_COUNT) as TOTAL_DISASTER_EVENTS,
    SUM(n.TOTAL_DAMAGE) as TOTAL_DAMAGE_COST,
    COUNT(DISTINCT n.SEVERE_EVENT_TYPE) as NUM_DISASTER_TYPES

FROM CLEANED_HPSA h
INNER JOIN CLEANED_CDC_SVI c ON h.FIPS_CODE = c.FIPS_CODE
INNER JOIN CLEANED_NOAA n ON h.FIPS_CODE = n.FIPS_CODE
GROUP BY h.FIPS_CODE, h.STATE_ABBR;

In [ ]:

-- Should be ~787 counties

In [ ]:
SELECT * FROM PROCESSED_DATA.JOINED_COUNTY_DATA LIMIT 10

Join by state with traffic

In [ ]:
-- state level traffic
CREATE OR REPLACE TABLE STATE_TRAFFIC_AVERAGES AS
SELECT 
    STATE_ABBR,
    AVG(TOTAL_CONGESTION_SCORE) as AVG_STATE_CONGESTION,
    AVG(TOTAL_WEATHER_CONGESTION) as AVG_STATE_WEATHER_CONGESTION,
    AVG(TOTAL_INCIDENT_CONGESTION) as AVG_STATE_INCIDENT_CONGESTION
FROM CLEANED_TRAFFIC
GROUP BY STATE_ABBR;


CREATE OR REPLACE TABLE JOINED_COUNTY_DATA AS
SELECT 
    h.FIPS_CODE,
    h.STATE_ABBR,
    
    -- HPSA Features (worst-case scenario)
    MAX(h.HPSA_SCORE) as MAX_HPSA_SCORE,
    AVG(h.HPSA_SCORE) as AVG_HPSA_SCORE,
    COUNT(DISTINCT h.DESIGNATION_TYPE) as NUM_DESIGNATION_TYPES,
    MAX(h.HPSA_DESIGNATION_POPULATION) as HPSA_POPULATION,
    MAX(h.PCT_POPULATION_BELOW_POVERTY) as POVERTY_RATE,
    MAX(h.RURAL_STATUS) as RURAL_STATUS,
    
    -- CDC SVI Features
    MAX(c.OVERALL_SVI_SCORE) as SVI_SCORE,
    MAX(c.SOCIOECONOMIC_VULNERABILITY) as SOCIOECONOMIC_SVI,
    MAX(c.HOUSEHOLD_VULNERABILITY) as HOUSEHOLD_SVI,
    MAX(c.MINORITY_STATUS) as MINORITY_SVI,
    MAX(c.HOUSING_TRANSPORT) as HOUSING_TRANSPORT_SVI,
    MAX(c.TOTAL_POPULATION) as TOTAL_POPULATION,
    
    -- NOAA Disaster Features
    SUM(n.EVENT_COUNT) as TOTAL_DISASTER_EVENTS,
    SUM(n.TOTAL_DAMAGE) as TOTAL_DAMAGE_COST,
    COUNT(DISTINCT n.SEVERE_EVENT_TYPE) as NUM_DISASTER_TYPES

FROM CLEANED_HPSA h
INNER JOIN CLEANED_CDC_SVI c ON h.FIPS_CODE = c.FIPS_CODE
INNER JOIN CLEANED_NOAA n ON h.FIPS_CODE = n.FIPS_CODE
GROUP BY h.FIPS_CODE, h.STATE_ABBR;


CREATE OR REPLACE TABLE FINAL_MASTER_DATASET AS
SELECT 
    j.*,
    
    -- State-level traffic features
    t.AVG_STATE_CONGESTION as STATE_TRAFFIC_CONGESTION,
    t.AVG_STATE_WEATHER_CONGESTION as STATE_WEATHER_TRAFFIC,
    t.AVG_STATE_INCIDENT_CONGESTION as STATE_INCIDENT_TRAFFIC
    
FROM JOINED_COUNTY_DATA j
LEFT JOIN STATE_TRAFFIC_AVERAGES t ON j.STATE_ABBR = t.STATE_ABBR;

In [ ]:
SELECT * FROM PROCESSED_DATA.FINAL_MASTER_DATASET LIMIT 20

In [ ]:
-- Check the final dataset
SELECT 
    COUNT(*) as total_counties,
    COUNT(DISTINCT STATE_ABBR) as states_represented,
    AVG(STATE_TRAFFIC_CONGESTION) as avg_traffic_score
FROM FINAL_MASTER_DATASET;

-- See traffic data by state
SELECT 
    STATE_ABBR,
    COUNT(*) as county_count,
    AVG(STATE_TRAFFIC_CONGESTION) as state_traffic_avg
FROM FINAL_MASTER_DATASET
GROUP BY STATE_ABBR
ORDER BY state_traffic_avg DESC;

Next steps: make final risk score/scores by normalizing


-Medical Risk = HPSA × SVI (county-level)
-Disaster Risk = Storm events (county-level)  
-Infrastructure Risk = Traffic congestion (state-level)
-Overall Risk = Weighted combination of all three

CREATING RISK SCORES + REGRESSION MODEL 

In [ ]:
# ===== REGRESSION MODEL WITH STORM FEATURES =====
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, coalesce, lit, lpad
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

session = get_active_session()

print(f" Current Location: {session.get_current_database()}.{session.get_current_schema()}")

# ===== LOAD FINAL MASTER DATASET =====
final_df = session.table("FINAL_MASTER_DATASET")
print(f" Loaded FINAL_MASTER_DATASET: {final_df.count()} counties")

# ===== LOAD STORM FEATURES FROM STORM MODEL =====
storm_features = session.table("NOAA_STORM_EVENTS_2025.PUBLIC.STORM_COUNTY_RISK_FEATURES")
print(f" Loaded STORM_COUNTY_RISK_FEATURES: {storm_features.count()} counties")

In [ ]:
# ===== ENSURE BOTH FIPS CODES ARE 5-DIGIT STRINGS =====
# clean final dataset FIPS
final_df = final_df.with_column(
    "FIPS_CODE_CLEAN",
    lpad(col("FIPS_CODE").cast("STRING"), lit(5), lit("0")) # ensure all codes are 5 digits with leading zeros
)

# clean storm features FIPS 
storm_features = storm_features.with_column(
    "FIPS_CODE_CLEAN",
    lpad(col("FIPS_CODE").cast("STRING"), lit(5), lit("0"))
)

# ===== JOIN ON 5-DIGIT FIPS =====
regression_df = final_df.join( # join Storm features 
    storm_features,
    final_df["FIPS_CODE_CLEAN"] == storm_features["FIPS_CODE_CLEAN"],
    join_type="left"
).drop(storm_features["FIPS_CODE"], storm_features["FIPS_CODE_CLEAN"])

print("Joined storm features at county level")

In [ ]:
# ===== FILL NULL VALUES FOR COUNTIES WITHOUT STORM DATA =====
storm_cols = [
    'STORM_SEVERITY_MEAN',
    'STORM_SEVERITY_MAX',
    'STORM_SEVERITY_STD',
    'STORM_INJURIES_TOTAL',
    'STORM_DEATHS_TOTAL',
    'STORM_DAMAGE_PROPERTY_TOTAL',
    'STORM_DAMAGE_CROPS_TOTAL',
    'STORM_EVENT_COUNT'
]
# for columns without storm data, the storm columns will be NULL
for col_name in storm_cols:
    regression_df = regression_df.with_column(
        col_name,
        coalesce(col(col_name), lit(0.0))
    )

In [ ]:
# ===== CONVERT TO PANDAS =====
regression_pandas = regression_df.to_pandas()
print(f" Converted to pandas. Shape: {regression_pandas.shape}")

# identify the FIPS column (it has a weird prefix from the join)
fips_col = [c for c in regression_pandas.columns if 'FIPS_CODE' in c and 'CLEAN' not in c][0]
print(f"\n FIPS column identified as: {fips_col}")

# rename it to something simple
regression_pandas = regression_pandas.rename(columns={fips_col: 'FIPS_CODE'})

# check storm data coverage
counties_with_storms = regression_pandas[regression_pandas['STORM_EVENT_COUNT'] > 0].shape[0]
total_counties = len(regression_pandas)
print(f"\n {counties_with_storms}/{total_counties} counties have storm history ({counties_with_storms/total_counties*100:.1f}%)")

# shows combined data 
print("\n📋 Sample of data with storm features:")
display_cols = ['FIPS_CODE', 'STATE_ABBR', 'STORM_EVENT_COUNT', 
                'STORM_SEVERITY_MEAN', 'STORM_DAMAGE_PROPERTY_TOTAL']
print(regression_pandas[display_cols].head(10))

In [ ]:
# ===== CREATE SEPARATE RISK SCORES =====

# ----- 1. MEDICAL RISK = HPSA × SVI -----
# Normalize each component first
scaler_medical = MinMaxScaler()
medical_components = regression_pandas[['MAX_HPSA_SCORE', 'SVI_SCORE']].fillna(0)
medical_normalized = scaler_medical.fit_transform(medical_components)

# Multiply HPSA × SVI (both normalized to 0-1)
regression_pandas['MEDICAL_RISK'] = (
    medical_normalized[:, 0] * medical_normalized[:, 1]
) * 100  # Scale to 0-100

print(f" Medical Risk created (HPSA × SVI)")
print(f"   Range: {regression_pandas['MEDICAL_RISK'].min():.2f} - {regression_pandas['MEDICAL_RISK'].max():.2f}")
print(f"   Mean: {regression_pandas['MEDICAL_RISK'].mean():.2f}")

# ----- 2. DISASTER RISK = Storm Model Predictions -----
# Use our storm features to create disaster risk
scaler_disaster = MinMaxScaler()
disaster_components = regression_pandas[[
    'STORM_SEVERITY_MEAN',
    'STORM_SEVERITY_MAX', 
    'STORM_EVENT_COUNT',
    'STORM_DAMAGE_PROPERTY_TOTAL',
    'STORM_INJURIES_TOTAL',
    'STORM_DEATHS_TOTAL'
]].fillna(0)
disaster_normalized = scaler_disaster.fit_transform(disaster_components)

# Weighted combination of storm features
disaster_weights = {
    'STORM_SEVERITY_MEAN': 0.30,           # How severe on average
    'STORM_SEVERITY_MAX': 0.20,            # Worst case scenario
    'STORM_EVENT_COUNT': 0.15,             # Frequency
    'STORM_DAMAGE_PROPERTY_TOTAL': 0.20,   # Economic impact
    'STORM_INJURIES_TOTAL': 0.10,          # Human impact
    'STORM_DEATHS_TOTAL': 0.05             # Fatality risk
}

# multiply each normalized storm feature by its weight
# add them all together, scale to 0-100
regression_pandas['DISASTER_RISK'] = (
    disaster_normalized[:, 0] * disaster_weights['STORM_SEVERITY_MEAN'] +
    disaster_normalized[:, 1] * disaster_weights['STORM_SEVERITY_MAX'] +
    disaster_normalized[:, 2] * disaster_weights['STORM_EVENT_COUNT'] +
    disaster_normalized[:, 3] * disaster_weights['STORM_DAMAGE_PROPERTY_TOTAL'] +
    disaster_normalized[:, 4] * disaster_weights['STORM_INJURIES_TOTAL'] +
    disaster_normalized[:, 5] * disaster_weights['STORM_DEATHS_TOTAL']
) * 100  # Scale to 0-100

print(f"\n Disaster Risk created (Storm Model Features)")
print(f"   Range: {regression_pandas['DISASTER_RISK'].min():.2f} - {regression_pandas['DISASTER_RISK'].max():.2f}")
print(f"   Mean: {regression_pandas['DISASTER_RISK'].mean():.2f}")

# ----- 3. INFRASTRUCTURE RISK = Traffic Congestion -----
# takes state-level traffic congestion score
# normalizes to 0-1
# scales to 0-100
scaler_infra = MinMaxScaler()
infra_components = regression_pandas[['STATE_TRAFFIC_CONGESTION']].fillna(0)
infra_normalized = scaler_infra.fit_transform(infra_components)

regression_pandas['INFRASTRUCTURE_RISK'] = infra_normalized[:, 0] * 100

print(f"\n Infrastructure Risk created (Traffic Congestion)")
print(f"   Range: {regression_pandas['INFRASTRUCTURE_RISK'].min():.2f} - {regression_pandas['INFRASTRUCTURE_RISK'].max():.2f}")
print(f"   Mean: {regression_pandas['INFRASTRUCTURE_RISK'].mean():.2f}")

# ----- 4. OVERALL RISK = Weighted Combination -----(
# current weights (ADJUST IF NECESSARY)
overall_weights = {
    'MEDICAL_RISK': 0.40,           # 40% weight
    'DISASTER_RISK': 0.35,          # 35% weight
    'INFRASTRUCTURE_RISK': 0.25     # 25% weight
}

regression_pandas['OVERALL_RISK'] = (
    (regression_pandas['MEDICAL_RISK'] / 100) * overall_weights['MEDICAL_RISK'] +
    (regression_pandas['DISASTER_RISK'] / 100) * overall_weights['DISASTER_RISK'] +
    (regression_pandas['INFRASTRUCTURE_RISK'] / 100) * overall_weights['INFRASTRUCTURE_RISK']
) * 100

print(f"\n Overall Risk created (Weighted Combination)")
print(f"   Weights: Medical {overall_weights['MEDICAL_RISK']:.0%}, Disaster {overall_weights['DISASTER_RISK']:.0%}, Infrastructure {overall_weights['INFRASTRUCTURE_RISK']:.0%}")
print(f"   Range: {regression_pandas['OVERALL_RISK'].min():.2f} - {regression_pandas['OVERALL_RISK'].max():.2f}")
print(f"   Mean: {regression_pandas['OVERALL_RISK'].mean():.2f}")

# ----- 5. SHOW DISTRIBUTION OF RISK SCORES -----
print("\n RISK SCORE DISTRIBUTIONS:")
print("\nMedical Risk:")
print(regression_pandas['MEDICAL_RISK'].describe())
print("\nDisaster Risk:")
print(regression_pandas['DISASTER_RISK'].describe())
print("\nInfrastructure Risk:")
print(regression_pandas['INFRASTRUCTURE_RISK'].describe())
print("\nOverall Risk:")
print(regression_pandas['OVERALL_RISK'].describe())

# ----- 6. SHOW SAMPLE COUNTIES -----
print("\n SAMPLE: Risk Breakdown by County")
sample_cols = ['FIPS_CODE', 'STATE_ABBR', 'MEDICAL_RISK', 'DISASTER_RISK', 
               'INFRASTRUCTURE_RISK', 'OVERALL_RISK']
print(regression_pandas[sample_cols].head(10).round(2).to_string(index=False))

print("="*60 + "\n")


Add some Exploratory Data Analysis

In [ ]:
print("\nBasic Statistics:")
display(regression_pandas.describe())

In [ ]:
# Manually select the key numeric features you want to correlate
key_numeric_features = [
    'MAX_HPSA_SCORE', 'AVG_HPSA_SCORE', 'SVI_SCORE', 'SOCIOECONOMIC_SVI',
    'HOUSEHOLD_SVI', 'MINORITY_SVI', 'HOUSING_TRANSPORT_SVI', 'POVERTY_RATE',
    'TOTAL_POPULATION', 'TOTAL_DISASTER_EVENTS', 'TOTAL_DAMAGE_COST', 
    'NUM_DISASTER_TYPES', 'STATE_TRAFFIC_CONGESTION', 'STATE_WEATHER_TRAFFIC',
    'STATE_INCIDENT_TRAFFIC'
]

# Filter to only include columns that actually exist in your dataframe
existing_features = [col for col in key_numeric_features if col in regression_pandas.columns]

plt.figure(figsize=(14, 10))
corr_matrix = regression_pandas[existing_features].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, square=True, cbar_kws={"shrink": .8})
plt.title('Key Features Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ===== PREPARE FEATURES FOR MODELING =====
# remove what we are trying to predict, so we don't use them as inputs
exclude_cols = [
    'FIPS_CODE', 
    'MEDICAL_RISK', 'DISASTER_RISK', 'INFRASTRUCTURE_RISK', 'OVERALL_RISK',  # These are targets/outputs
    'STATE_ABBR', 'RURAL_STATUS'
]


feature_cols = [col for col in regression_pandas.columns 
                if col not in exclude_cols and regression_pandas[col].dtype in ['int64', 'float64']]

# set features and label
# what we are predicting: overall risk from the features 
X = regression_pandas[feature_cols].fillna(0)
y = regression_pandas['OVERALL_RISK']  # ← Changed from RISK_SCORE to OVERALL_RISK

print(f"\n📋 Features in model:")
print(f"   Total features: {len(feature_cols)}")
print(f"   Storm features: {len([c for c in feature_cols if c.startswith('STORM_')])}")
print(f"   Target: OVERALL_RISK (combination of Medical, Disaster, Infrastructure)")

In [ ]:
# ===== TRAIN_TEST_SPLIT =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n Data split:")
print(f"   Training samples: {len(X_train)}")
print(f"   Testing samples: {len(X_test)}")

# ===== SCALE FEATURES =====
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===== TRAIN MODEL =====
print("\n Training Gradient Boosting Regressor...")

reg_model = GradientBoostingRegressor(
    n_estimators=100,      # Keep
    max_depth=3,           # ← Reduce to 3 (from 4)
    learning_rate=0.1,     # Keep
    random_state=42,
    subsample=0.8,
    min_samples_split=15,  # ← Increase to 15 (from 10)
    min_samples_leaf=7     # ← Increase to 7 (from 5)
)

reg_model.fit(X_train_scaled, y_train)
print(" Model training complete!")

# ===== EVALUATE MODEL =====
y_pred_train = reg_model.predict(X_train_scaled)
y_pred_test = reg_model.predict(X_test_scaled)

print("\n" + "="*60)
print(" REGRESSION MODEL PERFORMANCE")
print("="*60)
print(f"Train R² Score: {r2_score(y_train, y_pred_train):.4f}")
print(f"Test R² Score:  {r2_score(y_test, y_pred_test):.4f}")
print(f"Test RMSE:      {np.sqrt(mean_squared_error(y_test, y_pred_test)):.4f}")
print(f"Test MAE:       {mean_absolute_error(y_test, y_pred_test):.4f}")
print("="*60)

In [ ]:
# ===== FEATURE IMPORTANCE =====
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': reg_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n TOP 20 MOST IMPORTANT FEATURES:")
print(feature_importance.head(20).to_string(index=False))

# highlight storm features
storm_feature_importance = feature_importance[
    feature_importance['feature'].str.startswith('STORM_')
].reset_index(drop=True)

if len(storm_feature_importance) > 0:
    print("\n STORM FEATURE IMPORTANCE (from prediction model):")
    print(storm_feature_importance.to_string(index=False))
    
    total_storm_importance = storm_feature_importance['importance'].sum()
    print(f"\n Storm features contribute {total_storm_importance:.2%} to overall model importance")

In [ ]:
# ===== VISUALIZATIONS =====
# Plot 1: Feature Importance
plt.figure(figsize=(12, 8))
top_20 = feature_importance.head(20)
colors = ['red' if x.startswith('STORM_') else 'steelblue' for x in top_20['feature']]
plt.barh(range(len(top_20)), top_20['importance'], color=colors)
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Importance Score')
plt.title('Top 20 Most Important Features\n(Red = Storm Model Features)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Plot 2: Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_test, alpha=0.6, edgecolors='k', s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Risk Score', fontsize=12)
plt.ylabel('Predicted Risk Score', fontsize=12)
plt.title(f'Actual vs Predicted Risk Scores\nR² = {r2_score(y_test, y_pred_test):.4f}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Plot 3: Residuals
plt.figure(figsize=(10, 6))
residuals = y_test - y_pred_test
plt.scatter(y_pred_test, residuals, alpha=0.6, edgecolors='k', s=50)
plt.axhline(y=0, color='r', linestyle='--', lw=2, label='Zero Residual')
plt.xlabel('Predicted Risk Score', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.title('Residual Plot', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ===== SAVE PREDICTIONS BACK TO SNOWFLAKE =====
regression_pandas['PREDICTED_OVERALL_RISK'] = reg_model.predict(scaler.transform(X))

# create output dataframe with ALL risk scores
output_df = regression_pandas[[
    'FIPS_CODE', 
    'STATE_ABBR', 
    'MEDICAL_RISK',
    'DISASTER_RISK', 
    'INFRASTRUCTURE_RISK',
    'OVERALL_RISK',
    'PREDICTED_OVERALL_RISK'
] + storm_cols].copy()

# Convert back to Snowpark and save
output_sp = session.create_dataframe(output_df)
output_sp.write.mode("overwrite").save_as_table("COUNTY_RISK_PREDICTIONS_WITH_STORM")

print("\n Predictions saved to COUNTY_RISK_PREDICTIONS_WITH_STORM table")
print(f" Saved {len(output_df)} county risk predictions")
print("\n Output includes:")
print("   - MEDICAL_RISK (HPSA × SVI)")
print("   - DISASTER_RISK (Storm model predictions)")
print("   - INFRASTRUCTURE_RISK (Traffic congestion)")
print("   - OVERALL_RISK (Weighted combination)")
print("   - PREDICTED_OVERALL_RISK (ML model prediction)")
print("   - All 8 storm features")

In [ ]:
# ===== FINAL SUMMARY =====
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"Total counties analyzed: {total_counties}")
print(f"Counties with storm history: {counties_with_storms} ({counties_with_storms/total_counties*100:.1f}%)")
print(f"\nRisk Score Breakdown:")
print(f"   Medical Risk (HPSA × SVI): {regression_pandas['MEDICAL_RISK'].mean():.2f} avg")
print(f"   Disaster Risk (Storm Model): {regression_pandas['DISASTER_RISK'].mean():.2f} avg")
print(f"   Infrastructure Risk (Traffic): {regression_pandas['INFRASTRUCTURE_RISK'].mean():.2f} avg")
print(f"   Overall Risk (Combined): {regression_pandas['OVERALL_RISK'].mean():.2f} avg")

# Show top 10 highest risk counties (for stakeholders)
print("\n TOP 10 HIGHEST RISK COUNTIES:")
top_risk_cols = ['FIPS_CODE', 'STATE_ABBR', 'OVERALL_RISK', 'MEDICAL_RISK', 
                 'DISASTER_RISK', 'INFRASTRUCTURE_RISK']
top_risk = regression_pandas.nlargest(10, 'OVERALL_RISK')[top_risk_cols]
print(top_risk.round(2).to_string(index=False))

print("\n Analysis complete!")